In [2]:
import torch
import torch.nn as nn
import math

## 总览

模块包含 : 

- InputEmbeddings
- PositionalEncoding
- LayerNormalization
- FeedForwardBlock
- MultiHeadAttentionBlock
- ResidualConnection
- EncoderBlock
- Encoder
- DecoderBlock
- Decoder
- ProjectionLayer
- Transformer

模块拼装 :

``` mermaid
Transformer
 ├── Encoder
 │    ├── InputEmbeddings + PositionalEncoding
 │    └── EncoderBlock x N
 |        ├── ResidualConnection   
 │        │    ├── MultiHeadAttentionBlock (Self Attention)
 │        │    └── FeedForwardBlock
 │        └── LayerNormalization
 │         
 ├── Decoder
 │    ├── InputEmbeddings + PositionalEncoding
 │    └── DecoderBlock x N
 |        ├── ResidualConnection   
 │        │    ├── MultiHeadAttentionBlock (Masked)
 │        │    ├── MultiHeadAttentionBlock (Cross Attention)
 │        │    └── FeedForwardBlock
 │        └── LayerNormalization
 └── ProjectionLayer
 ```

## Str -> Vec

首先,对原始语言的处理分为多步 : 首先将句子切成 tokens ; 然后(更主流一些地 ,) 根据数据集构造字符到索引的字典 ( 学的时候输入 int 而不是 str );

( 字典也能由人工根据语言设计 , 但不是主流 )

再将每个索引 int 映射到向量空间中 , 变成一个 vec .

字典构造一般技巧比较固定 , 因而可以直接调对应轮子 .

它们的实现由于与模型本身耦合不大 , 所以被放在了dataset的构建里

### InputEmbeddings

而 InputEmbedding 层完成的是 int -> vec 的这一嵌入步骤 , 也就是说它输入的 x 是一个 int 构成的 tensor , 输出的是一个 ( seq_len , d_model ) 的向量 , 再加上 batch_size , 得到这里的维度转换: 

**( batch_size , seq_len ) --> ( batch_size , seq_len , d_model )**

理解为有 vocab_size 个词 , 它们的语义关系将在 d_model 维度的空间中得以表现 . 而按照原论文 , 这里 d_model 取512

而 nn.Embedding 层虽然每次面向的是不同的 minibatch , 却能保证同 id 的token 被映射到空间中的同一位置



In [3]:
class InputEmbeddings(nn.Module):
    
    def __init__(self, d_model: int, vocab_size: int) -> None:
        super().__init__()
        self.d_model = d_model
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(vocab_size, d_model)
        

    def forward(self, x):
        return self.embedding(x) * math.sqrt(self.d_model)

### PositionalEncoding

考虑 Transformer 的计算过程 : q , k , v 三者之间都会互相遇到计算 ( 在 encoder 中因为是自注意力 , 这一对称表现更为明显 ) , 所以输入的虽然是一个有 index 这一信息的 tensor , 却实际上需要人为地将位置信息编码进里面 . 

对于一个尺寸为 ( batch_size , seq_len , d_model ) 的输入 , 从第 0 维中取一个 (seq_len , len) 尺寸的切片即位一个句子单位 , 也就是说也就是说位置编码的作用效果应该在其上表示出来 , 而第 1 维 seq_len 中的每一个也就是一个 token , 也就是说位置信息应该加到最后的 d_model 维度上 , 而且显然最好能 d_model 的每一位都能加些东西上去 . 

编码的方法有很多种 , 比如可以直接对 d_model 这一行的数值做修改 , 也可以加 d_model 的维度 . 原论文中选用了基于三角函数的实现 , 具体上是修改数值 . 

论文位置编码 : 
$$PE_{(pos,2i)}=\sin{\frac{pos}{10000^{2i/d_{model}}}}$$

$$PE_{(pos,2i+1)}=\cos{\frac{pos}{10000^{2i/d_{model}}}}$$

为什么要用三角函数 ? 一个说法是其 $\sin(x+y) = \sin(x)\cos(y) + \cos(x)\sin(y)$ 之类的式子方便将相对关系 ( 比如 x + h ) 变成线性计算 .

为什么可以通过改数值来传递位置信息 ? ( 而不会污染原信息 , 或无法正常传入信息 ? ) : 因为这种编码加入是一直进行 , 而不是随机选某一次加入的 . 比如一个照片固定中心上有一个黑点 , 的确会使单张图片被挡住一些内容 , 但只要照片够多 , 每次都拍物体的某一部分 , 也能最终正常表现物体 , 而其上有没有黑点就是它是否在中央的编码信息 . ( 差不多是这种感觉 )

而代码不好理解的多是维度操作与切片操作 : 
- 首先创建一个 ( seq_len , d_model ) 大小的空张量
- 然后创建位置张量 : arange() 创建的是一个 0 到 n - 1 的向量 ( 尺寸为 ( seq_len, ) ) ; 由于需要相当于将其在 d_model 上铺成一排 , 为了方便广播 , 这里用 unsqueeze(1) , 意思是在第一维插入一个维度 , 变成 ( seq_len , 1 )
- div_term 的 `arange(0, d_model, 2)` 指的是 0 到 d_model - 1 , 步长为2 ( 这里能保证得到一个有空隙的错开向量 , 方便后面分别两个间隔里放 sin 与 cos ) . 而对一个 tensor 进行标量尺度的操作 ( 比如 `* (-math.log(10000.0) / d_model)` , 相当于对其中每个元素做对应操作 )
- 接下来 position * div_term 尺寸 : ( seq_len , 1 ) * ( d_model / 2 , ) 可以发生广播 , 变成 ( seq_len , d_model / 2 ) ( 注意这里对 position , div_term 二者其一要进行维度拓展 , 不然两个向量相乘不知道是应该广播还是点乘积 . ) 而计算中 , 前者表示有 seq_len 行 , 每行有一个元素 , 值为当前行数减一 , 广播后 , 每行有相同值的 d_model / 2 个元素 , 与 div_term 相乘 
- `pe[:, 0::2]`积累跳跃地选取方式
- `pe = pe.unsqueeze(0)`相当于补上 batch_size 维度
- 通过 register_buffer 将这个位置编码向量注册到 module 的 buffer 而非 parameters 里 . 这是因为它需要被保存参量时也被存下来 , 且可以参与 forward 等计算过程 , 以及 .to(device) 等模型的迁移过程 , 但不需要更新 , 不需要被 optimizer 注意到 .

- 而在 forward 函数中 , 又涉及到 `[:, :x.shape[1], :]` 的切片 , 可以发现这里的切片主要发生在 seq_len 所在维度 , 它截取到 x 的长度 . 注意这里的 x 仍然为一批的 seq , 但是它的 seq_len 是可以动态调整的 , 而 PositionalEncoding 模块因为只用实例化一次 , 输入的 seq_len 是数据集的最大 seq_len , 所以有进行截取的必要性

P.S. : 关于unsqueeze(n) : 在第 n 维度插入新维度 , 比如源码该位置若改为 unsqueeze(0) , 就变成尺寸 ( 1 , seq_len ) 


In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, seq_len: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model
        self.seq_len = seq_len
        self.dropout = nn.Dropout(dropout)
        
        pe = torch.zeros(seq_len, d_model)
        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1) 
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)) 
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + (self.pe[:, :x.shape[1], :]).requires_grad_(False) 
        return self.dropout(x)

## Vec 处理

### LayerNormalization 

LayerNorm 相对应的概念是 BatchNorm . 后者选择的是同一批量内的元素的同一特征进行归一化 , 但是在这里将会是对 token 进行对齐正则 , 这在不同长度的句子里是无意义的 . 而前者则是一个句子内部正则化 , 相当于记录原句子的起伏 , 但缩放其极值 .

( BN 在 CNN 中较为常用 , 是因为其暗示了每个尺度 ( 一般来说是 channel ) 的分布是有某种类似性的 , 这是不同句子间很难具有的 ) 

Normalization 不只是设置均值为零 , 方差为一 , 而是在这个值的基础上继续一次线性变换 , 来学系数与偏置 ( 最差也是学到分别为一与零 , 至少不会效果倒退 ) , 这在所有的 Normalization 里基本都是适用的 .

eps 即 $\epsilon$ , 防止原方差为 0 导致除法出错的极小量 .

使用 `nn.Parameter(tensor)` 声明参量可以将它放到 `nn.Module` 的参数列表里 , 从而达到能被优化器更新的效果 .

但这不代表它和对 `torch.Tensor` 设置 `requires_grad = True` 是一个东西 , 后者不会出现在 `model.parameters()` 里 , 从而 optimizer 不会更新它 .

torch 中声明 `requires_grad = True` 的 Tensor 都会计算梯度 , 但基于梯度的 `optimizer.step()` 只会更新被注册到 `model.parameters()` 里的参量 .


`dim=-1` 代表取最后一个维度 , 这里 x 最后一个维度是 batch_size , 也就是说它会每一个样本内部每个维度算自己的 mean 与 std , 也就是 layer norm

而计算后会有维度压缩 , 被压缩的维度直接消失 ; keepdim 可以使得这个维度被保留且长度为 1 . 比如 mean ( keepdim = False ) --> ( batch_size , ) , 但 True 时为( batch_size , 1 ) . 考虑到这两个量后面要和 x 自己做正则化运算的 , 这里 keepdim 方便广播

( 2 , ) 与 ( 2 , 1 ) 不是一个东西 , 这个表现在比如解释性上 ( 比如前者是行向量还是列向量 ? ) . ( 更本质地 , 表现在二者代表的 python tuple 的形状上 )

#### 补 : 关于参量 features

视频带着写的代码与仓库提供的代码在 model.py 中的一大区别就是后者多了 features 参量 . 虽然到处都是 , 但是追踪下发现它唯一作用到的地方就是 LN 层 . 
源代码声明 alpha 与 beta 时应形如 
```python
self.alpha = nn.Parameter(torch.tensor(1.0))
self.bias  = nn.Parameter(torch.tensor(0.0))
```
而这里将其从单个标量改为了与 x 尺寸存在匹配的 tensor , 具体来讲 , 它会在后面的 `build_transformer` 函数中被赋值为 d_model . 也就是说 token 的每一个语义维度都将得到一套自己的正则化参数 , 这显然有利于提高表现 .

In [ ]:
class LayerNormalization(nn.Module):

    def __init__(self, features: int, eps: float = 1e-6) -> None:
        super().__init__()
        self.eps = eps

        self.alpha = nn.Parameter(torch.ones(features)) # For Multiplication
        self.bias = nn.Parameter(torch.zeros(features)) # For Addition

    def forward(self, x: torch.Tensor):

        mean = x.mean(dim=-1, keepdim=True)
        std = x.std(dim=-1, keepdim=True)

        return self.alpha * (x - mean) / (std + self.eps) + self.bias


### FeedForwardBlock

很简单的 MLP 

可以注意的是 `nn.Linear` 层本身就包含了 bias , 不用看着论文公式再手动加 .

维度变化 : 
```mermaid
    (batch_size, seq_len, d_model) 
--> (batch_size, seq_len, d_ff) 
--> (batch_size, seq_len, d_model)
```

In [ ]:
class FeedForwardBlock(nn.Module):

    def __init__(self, d_model: int, d_ff : int, dropout: float) -> None:
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff) 
        self.linear2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()

    def forward(self, x: torch.Tensor):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        return x

### MultiHeadAttentionBLock

主要就是 k q v 三个向量以及其对应的操作矩阵 , 而能发现无论是 Encoder 还是 Decoder , 始终都是 k 与 q 来自同一者 ( 比如自注意力中三者全部来自 x , 而交叉注意力中前二者来自 Inputs , 后者来自 Outputs ) , 且二者发生的运算也仅有他们自己的互乘 , 也就是说 , 三个向量完全可以简化成一个向量 ( value ) 与一个矩阵 ( k , q 乘积 ) . 不这样做一方面是维持矩阵的可被拆解为两向量乘积的特征 , 另一方面是文章确实是这么写的 .

或许可以提供一种感觉 ? query 对 key 的查询一般来讲 ( 比如数据库 ) 是 " 硬 " 的 ( 这里的软硬可以联想分类问题 , 一个返回唯一确定值 , 一个返回全体的概率 , 需要进一步用取最大得到唯一值 ) , 而在 attention 这里是软的 , 是对每一个 key 做相当于加权平均的操作 .

而这三者事实上都来自于输入 x , 故 x 虽然没有直接出现在 forward 输入中 , 但也与线性层传参 x 一个性质 . 

而 python 类中可以用 `@staticmethod` 声明静态方法 , 这样就能在类里面直接用而不用实例化 .

在基础的注意力的想法上 , 进一步有了多头注意力的策略 : 将 d_model 的整片语义维度切分 , 让不同的注意力头去关注部分的维度 ( 可以类比成 channel ) . 又因为整个过程可以理解为为原 k q v 加维度 , 也就是说这些头的计算是可以在一次张量积算里并行算掉的 . 

而这里层级排序通过 transpose 进一步调整 : 因为需要让多出来的头 h 维度中 , 每一个都包含一个句子以及部分语义维度 . 

调整矩阵大小有 reshape 与 view 两种方法 , 后者特别作用于在内存中连续存储的数据 , 因为如同名字 , 只是更改系统操作内存时采用的视角而不是真的更改具体排布 , 因而带来性能优势 .

后面有与 view 搭配的 contiguous 方法 . 其作用是将可能不在内存中连续排列的数据改成连续排的 , 来方便 view 使用 .

另 , attention 中使用的 mask_fill_ 可以原地将 mask 中符合条件的填为 1 , 其他的填为 0 .

附带返回 attention score , 可以方便后面可视化之类的工作 . 

以下是具体的被操作维度与其互相转换关系 : 

attention:
- 输入 q , k , v : ( batch_size , h , seq_len , d_k )
- query @ key ^ T : ( batch_size , h , seq_len , d_k ) @ ( batch_size , h , d_k , seq_len ) --> ( batch_size , h , seq_len , seq_len )
- masked_fill : ( batch_size , h , seq_len , seq_len ) --> ( batch_size , h , seq_len , seq_len )
- attention_score @ value : ( batch_size , h , seq_len , seq_len ) @ ( batch_size , h , seq_len , d_k ) --> ( batch_size , h , seq_len , d_k )

- 返回 : ( batch_size , h , seq_len , d_k ) , ( batch_size , h , seq_len , seq_len )

forward : 
- 更新 k q v : ( batch_size , seq_len , d_model ) --> ( batch_size , seq_len , d_model )
- 为三者切分注意力头 : ( batch_size , seq_len , d_model ) --> ( batch_size , h , d_k ) --> ( batch , h , seq_len , d_k )

- x 重塑 : ( batch_size , h , seq_len , d_k ) --> ( batch_size , seq_len , h , d_k ) --> ( batch_size , seq_len , d_model )
- 经过 W_o 矩阵 : ( batch_size , seq_len , d_model ) --> ( batch_size , seq_len , d_model )

( 这部分补充笔记建议看过后面的 dataset.py 的相关笔记后 , 在大后期再来 , 结合着看会更清楚些 )

**在训练时** , decoder 是如何实现逐字符预测的效果的 ? 

其并不是像流一样 , 预测完一个再去下一个 , 而是一次性都预测掉 . mask 使得某一行的某位 token 之后的值全部不可见 , 使得后面的 token 参与乘积得到的结果全是 0 , 事实上等效于 " 这个句子结束于此 " . 而这个 token 将会逐个被扮演 , 来达到并行的 " 逐个预测 " 效果 . 

因此 , 训练时无需显示地所谓 " 一点点传入句子 "

但这与预测环节的过程**完全不一样** , 具体参见 train.py 的预测部分 . 这个名字相同 , 其他的基本都不同其实也是导致 decoder 部分不好搞懂的原因 .  

In [ ]:
class MultiHeadAttentionBlock(nn.Module):

    def __init__(self, d_model: int, h: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model
        self.h = h
        assert d_model % h == 0, "d_model should be divisible by h"

        self.d_k = d_model // h
        self.w_q = nn.Linear(d_model, d_model, bias=False)
        self.w_k = nn.Linear(d_model, d_model, bias=False)
        self.w_v = nn.Linear(d_model, d_model, bias=False)
        self.w_o = nn.Linear(d_model, d_model, bias=False) 
        self.dropout = nn.Dropout(dropout)

    @staticmethod
    def attention(query: torch.Tensor, key: torch.Tensor, value: torch.Tensor, mask: torch.Tensor, dropout: nn.Dropout):
        d_k = query.shape[-1]

        attention_score = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)
        if mask is not None:
            attention_score.masked_fill_(mask == 0, -1e9)

        attention_score = attention_score.softmax(dim=-1)
        if dropout is not None:
            attention_score = dropout(attention_score)

        return (attention_score @ value), attention_score

    def forward(self, q, k, v, mask):

        query = self.w_q(q) 
        key = self.w_k(k)
        value = self.w_v(v)

        query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1, 2)
        key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1, 2)
        value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1, 2)

        x, self.attention_scores = MultiHeadAttentionBlock.attention(query, key, value, mask, self.dropout)

        x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.h * self.d_k)

        return self.w_o(x)

### ResidualConnection

层本身在知道 Residual 相关知识后不难理解 , 值得注意的是一些实现细节 .

在 forward 函数中 , 传入的 sublayer 到底是什么 ? 这个可能在后面遇到其被用了 lambda 表达式后产生疑问 . 事实上其类别应该是 `<class 'function'>` 而非类名 . 这是因为lambda 对接的实际上是 `__call__()` 方法

而这还暗示一个重要的问题 : 这里传入的是对象实例而非通过实例计算得到的结果 . 后面遇到时会进一步分析 .  

另 , 同在 forward 函数中 , 选择先进行 sublayer 操作还是先 norm 操作是比较经验性的 , 论文里是与这里相反的 , 但这样先 sublayer 再 norm 的顺序在之后的实现中更常见 . 

In [ ]:
class ResidualConnection(nn.Module):

    def __init__(self, features: int, dropout: float) -> None:
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.norm = LayerNormalization(features)
    
    
    def forward(self, x: torch.Tensor, sublayer):
        return x + self.dropout(sublayer(self.norm(x)))

### EncoderBlock 

其相当于对用残差 ( 以及经过时的正则化 ) 连起来的注意力层与前馈层进行一个封装 , 来满足堆叠 N 个的需要 .

与 Encoder 名字很像 , 注意别搞混二者各自是什么 . 

`nn.ModuleList()` 正如前面需要 parameters 相关的注册 , 这里由于 self 的赋值对象是 list 而需要额外将其中的模块注册到 module 上 . 具体可以看 appendix 中的相关内容 . 

这里用的 lambda 表达式是构建了一个 k q v 输入都是 x 本身的类实例 ( 但是本身作为类的 forward 函数的输入的 'x' 仍然是不存在的 , 不要让这里的 x 与待计算的 x 混淆 , 而是将其理解为一个四元函数 , 传入被视为参量位置的恰好都是 x 本身 , 而刚好在这个函数里第四个元为实际上空的 )

In [ ]:
class EncoderBlock(nn.Module):

    def __init__(self,features: int, self_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock,
                 dropout: float) -> None:
        super().__init__()
        self.self_attention_block = self_attention_block
        self.feed_forward_block = feed_forward_block
        self.dropout = nn.Dropout(dropout)
        self.residual_connection = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(2)])
    
    def forward(self, x, src_mask):
        x = self.residual_connection[0](x, lambda x: self.self_attention_block(x, x, x, src_mask))
        x = self.residual_connection[1](x, self.feed_forward_block)
        return x

### Encoder

因为前面的模块已经搭好了 , 组装就很简单了

这里用了 Modulelist , 注意不是 sequential , 二者灵活性等没法比 , 应用场景不同 .

In [ ]:
class Encoder(nn.Module):

    def __init__(self, features: int, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization(features)

    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

### DecoderBlock 

decoder 最不同的地方是用了交叉注意力与遮罩 , 但它们的模型基础架构都没有变 , 直接改传参就能用 .

In [ ]:
class DecoderBlock(nn.Module):

    def __init__(self, features: int, self_attention_block: MultiHeadAttentionBlock, cross_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float):
        super().__init__()
        self.self_attention_block = self_attention_block
        self.cross_attention_block = cross_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(3)])

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, tgt_mask))
        x = self.residual_connections[1](x, lambda x: self.self_attention_block(x, encoder_output, encoder_output, src_mask))
        x = self.residual_connections[2](x, self.feed_forward_block)
        return x

### Decoder
类似 Encoder

In [ ]:
class Decoder(nn.Module):

    def __init__(self, features: int, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization(features)

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, encoder_output, src_mask, tgt_mask)
        return self.norm(x)

### ProjectionLayer

一个单层线性层 , 值得注意的是最后处理 : log_softmax 与 softmax 的区别仅仅在于外面多了层 log , 但这样可以防止

而如果训练时用了 CrossEntropyLoss , 返回时最后一层就不应该再用 ( log ) softmax , 这是因为 loss 计算时会自动对输入再来一次 log_softmax 导致计算重复 . 这里代码用了交叉熵所以直接返回线性层结果了 . 

( 也有最后一层用 log_softmax , 计算 loss 时再用 NLL loss 的方法 , 二者一起起到的效果完全与只用交叉熵相同 , 而预测阶段直接用 argmax , 也不用softmax 再正则化一遍 )

In [ ]:
class ProjectionLayer(nn.Module):

    def __init__(self, d_model: int, vocab_size: int) -> None:
        super().__init__()
        self.proj =  nn.Linear(d_model, vocab_size)
    
    def forward(self, x):
        # return torch.log_softmax(self.proj(x), dim=-1)
        return self.proj(x)

## Transformer 

其实也是将造好的子模块拼在一起 .

但其与接下来的 build_transformer 一起可以作为一个值得学习的思路 : 首先用类本身构建计算流程 , 但不要在取用时直接调用其构造函数 , 而是再封装一个构造函数 .

否则调用构造函数传参时会需要在括号里手动设置每个子模块的构造 , 也就是说会出现大量括号嵌套的情况 ( 其实这里甚至仅靠括号嵌套都几乎做不到 ) , 这是不应该暴露给使用者的 , 因为对于其来说这些需要手动设置的参数之间应该是平等的 , 而非还要记住隶属于哪个子模块中 .

另一方面 , 也有应该接触的层级划分有关 : 尽量不跨层干预构造等操作 . 

可以看到 build_transformer 中对外提供的参量就满足了刚刚的需求 . 

In [ ]:
class Transformer(nn.Module):

    def __init__(self, encoder: Encoder,
                 decoder: Decoder,
                 src_embed: InputEmbeddings,
                 tgt_embed: InputEmbeddings,
                 src_pos: PositionalEncoding,
                 tgt_pos: PositionalEncoding,
                 projection_layer: ProjectionLayer):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.src_pos = src_pos
        self.tgt_pos = tgt_pos
        self.project_layer = projection_layer

    def encode(self, src, src_mask):
        src = self.src_embed(src)
        src = self.src_pos(src)
        return self.encoder(src, src_mask)
    
    def decode(self, encoder_output: torch.Tensor, src_mask: torch.Tensor, tgt: torch.Tensor, tgt_mask: torch.Tensor):
        tgt = self.tgt_embed(tgt)
        tgt = self.tgt_pos(tgt)
        return self.decoder(tgt, encoder_output, src_mask, tgt_mask)

    def project(self, x):
        return self.project_layer(x)

In [ ]:
def build_transformer(src_vocab_size: int, 
                      tgt_vocab_size: int, 
                      src_seq_len: int,
                      tgt_seq_len: int, 
                      d_model: int = 512,
                      N: int = 6,
                      h: int = 8,
                      dropout: float = 0.1, 
                      d_ff = 2048) -> Transformer:
    src_embed = InputEmbeddings(d_model, src_vocab_size)
    tgt_embed = InputEmbeddings(d_model, tgt_vocab_size)

    src_pos = PositionalEncoding(d_model, src_seq_len, dropout)
    tgt_pos = PositionalEncoding(d_model, tgt_seq_len, dropout)

    encoder_blocks = []
    for _ in range(N):
        encoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        encoder_block = EncoderBlock(d_model, encoder_self_attention_block, feed_forward_block, dropout)
        encoder_blocks.append(encoder_block)

    decoder_blocks = []
    for _ in range(N):
        decoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        decoder_cross_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        decoder_block = DecoderBlock(d_model, decoder_self_attention_block, decoder_cross_attention_block, feed_forward_block, dropout)
        decoder_blocks.append(decoder_block)

    encoder = Encoder(d_model, nn.ModuleList(encoder_blocks))
    decoder = Decoder(d_model, nn.ModuleList(decoder_blocks))
    
    projection_layer = ProjectionLayer(d_model, tgt_vocab_size)

    transformer = Transformer(encoder, decoder, src_embed, tgt_embed, src_pos,tgt_pos, projection_layer)

    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

    return transformer

## 补充 

### 关于 Dropout

虽然 dropout 本身原理以及其使用的有效性不是讨论重点 , 但还是总结下其使用的地方 : 

- 嵌入后 ( InputEmbedding 以及 PositionalEncoding 后 ) 
- FeedForward 中间层
- attention_score 计算
- 残差层 : 对 f ( x ) 而非 x + f ( x ) 做 dropout
- EncoderBlock ( DecoderBlock ) 之间